# Assignment 3 Materials: Artificial Hi-C Contact Map Generator

This notebook provides tools for generating artificial Hi-C contact maps based on GWAS SNP data. The generated maps simulate:
- Distance-dependent contact decay (polymer physics)
- TAD-like domain structures
- SNP-driven chromatin loop enhancements


## Imports


In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist


## Hi-C Contact Map Generator Function

The `generate_artificial_hic` function creates a simulated Hi-C contact matrix for a genomic region defined by GWAS SNPs.

**Parameters:**
- `gwas_df`: DataFrame with GWAS summary data (requires `chromosome`, `position`, `odds_ratio` columns)
- `region_padding`: Base pairs added upstream/downstream of SNP span (default: 500,000)
- `bin_size`: Hi-C bin resolution in base pairs (default: 10,000)
- `base_decay`: Controls distance-dependent contact decay
- `snp_loop_strength`: Multiplicative boost for contacts near GWAS SNPs
- `tads`: Whether to simulate TAD-like domains
- `noise_level`: Gaussian noise magnitude

**Returns:**
- `hic_matrix`: Simulated Hi-C contact matrix
- `bin_positions`: Genomic coordinates of bin centers
- `metadata`: Information about region and modeling assumptions


In [ ]:
def generate_artificial_hic(
    gwas_df: pd.DataFrame,
    region_padding: int = 500_000,
    bin_size: int = 10_000,
    base_decay: float = 1.0,
    snp_loop_strength: float = 2.5,
    tads: bool = True,
    noise_level: float = 0.05,
    random_state: int = 42
):
    """
    Generate an artificial Hi-C contact map for a genomic region defined by GWAS SNPs.

    Parameters
    ----------
    gwas_df : pd.DataFrame
        GWAS summary table (see required columns below).

    region_padding : int
        Number of base pairs added upstream/downstream of SNP span.

    bin_size : int
        Hi-C bin resolution in base pairs.

    base_decay : float
        Controls distance-dependent contact decay.

    snp_loop_strength : float
        Multiplicative boost for contacts near GWAS SNPs.

    tads : bool
        Whether to simulate TAD-like domains.

    noise_level : float
        Gaussian noise magnitude.

    random_state : int
        Random seed.

    Returns
    -------
    hic_matrix : np.ndarray
        Simulated Hi-C contact matrix.

    bin_positions : np.ndarray
        Genomic coordinates of bin centers.

    metadata : dict
        Information about region and modeling assumptions.
    """

    np.random.seed(random_state)

    # ----------------------------
    # Define genomic region
    # ----------------------------
    chrom = gwas_df["chromosome"].iloc[0]
    start = gwas_df["position"].min() - region_padding
    end = gwas_df["position"].max() + region_padding

    bins = np.arange(start, end, bin_size)
    bin_centers = bins + bin_size // 2
    n_bins = len(bin_centers)

    # ----------------------------
    # Base polymer decay
    # ----------------------------
    distances = cdist(bin_centers[:, None], bin_centers[:, None], metric="euclidean")
    hic = np.exp(-distances / (bin_size * base_decay))

    # ----------------------------
    # Add TAD structure (optional)
    # ----------------------------
    if tads:
        tad_size = int(200_000 / bin_size)  # ~200kb TADs
        for i in range(0, n_bins, tad_size):
            hic[i:i+tad_size, i:i+tad_size] *= 1.5

    # ----------------------------
    # SNP-driven loop boosts
    # ----------------------------
    for _, snp in gwas_df.iterrows():
        snp_bin = np.argmin(np.abs(bin_centers - snp["position"]))
        effect = np.log1p(snp["odds_ratio"] - 1) * snp_loop_strength

        for j in range(n_bins):
            hic[snp_bin, j] *= (1 + effect)
            hic[j, snp_bin] *= (1 + effect)

    # ----------------------------
    # Add noise
    # ----------------------------
    noise = np.random.normal(0, noise_level, size=hic.shape)
    hic += noise
    hic[hic < 0] = 0

    # ----------------------------
    # Metadata
    # ----------------------------
    metadata = {
        "chromosome": chrom,
        "region_start": start,
        "region_end": end,
        "bin_size": bin_size,
        "n_bins": n_bins,
        "assumptions": {
            "distance_decay": True,
            "tads": tads,
            "snp_loops": True,
            "noise": True
        }
    }

    return hic, bin_centers, metadata


## Example Usage

Create a sample GWAS DataFrame and generate an artificial Hi-C contact map.


In [ ]:
# Example GWAS data
gwas_df = pd.DataFrame({
    "rsid": ["rs212388", "rs12345"],
    "chromosome": ["6", "6"],
    "position": [159069404, 159300000],
    "risk_allele": ["C", "T"],
    "odds_ratio": [1.105, 1.25],
    "pvalue": [3e-14, 1e-8],
    "risk_frequency": [0.41, 0.22]
})

gwas_df


In [ ]:
# Generate artificial Hi-C contact map
HiC_map, bin_centers, metadata = generate_artificial_hic(gwas_df)

print(f"Hi-C matrix shape: {HiC_map.shape}")
print(f"Number of bins: {len(bin_centers)}")
print(f"\nMetadata:")
metadata


## Visualization

Visualize the generated Hi-C contact map as a heatmap.


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(HiC_map, cmap='YlOrRd', aspect='equal')
ax.set_xlabel('Bin index')
ax.set_ylabel('Bin index')
ax.set_title(f'Artificial Hi-C Contact Map\nChromosome {metadata["chromosome"]}: {metadata["region_start"]:,} - {metadata["region_end"]:,}')
plt.colorbar(im, ax=ax, label='Contact frequency')
plt.tight_layout()
plt.show()
